In [4]:
import base64
from datetime import date, datetime
import json
import os
import random
import re
import time

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook
%matplotlib inline
%config InlineBackend.figure_format='retina'

from IPython.display import display
from tqdm import tqdm

# Set random seed for reproducibility
def set_seeds(seed=777):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

SEED = 777
set_seeds(SEED)
print(f'Random seed set to: {SEED}')
# Checking
print(np.random.rand(3))
set_seeds(SEED)
print(np.random.rand(3))

# Display entire DataFrame
def print_all(df):
    with pd.option_context('display.max_rows', None, 
                           'display.max_columns', None, 
                           'display.float_format', '{:,.4f}'.format):
        display(df)
# Display entire columns
def print_cols(df, n=5): 
    with pd.option_context('display.max_columns', None,
                          'display.float_format', '{:,.4f}'.format):
        print(df.shape)
        display(df[:n])

Random seed set to: 777
[0.15266373 0.30235661 0.06203641]
[0.15266373 0.30235661 0.06203641]


In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score, log_loss

In [6]:
# ===================================================================
# Configurations

# Path
# Get the path of the current script file
CURRENT_DIR = os.getcwd()

# Move up one level to the parent directory
PARENT_DIR = os.path.dirname(os.getcwd())

# Path to data directory
DATA_ROOT = os.path.join(CURRENT_DIR, 'data')

# Path to dataset files
TRAIN_DATA_PATH = os.path.join(DATA_ROOT, 'train_scaled.csv')
TEST_DATA_PATH = os.path.join(DATA_ROOT, 'test_scaled.csv')

In [8]:
# target column
target_col = 'def_pay'

# Data Load

In [9]:
# Load dataset
train_df = pd.read_csv(TRAIN_DATA_PATH)
test_df = pd.read_csv(TEST_DATA_PATH)

train_df[target_col] = train_df[target_col].astype('category')
test_df[target_col] = test_df[target_col].astype('category')

print_cols(train_df, 2)
print_cols(test_df, 2)

(24000, 27)


,limit_bal,sex_female,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,edu_1,edu_2,edu_3,marr_1,marr_2,def_pay
0,-0.6761,1,-1.1345,-0.4695,-0.4000,-0.3889,-0.3446,-0.3127,-0.3195,-0.0815,-0.0624,-0.0387,0.0062,0.0393,0.0865,-0.2138,-0.1676,-0.1818,-0.2166,-0.0966,-0.2932,0,1,0,0,1,0
1,0.9401,1,1.4672,-0.4695,-0.4000,-0.3889,-0.3446,-0.3127,-0.3195,0.0198,0.0663,0.0990,0.1729,0.2424,0.2854,-0.1873,-0.1752,-0.1831,-0.1844,-0.1866,-0.1795,0,1,0,0,0,0


(6000, 27)


,limit_bal,sex_female,age,pay_1,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,bill_amt2,bill_amt3,bill_amt4,bill_amt5,bill_amt6,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,edu_1,edu_2,edu_3,marr_1,marr_2,def_pay
0,-0.1374,1,-0.9177,-0.4695,-0.4000,-0.3889,-0.3446,-0.3127,-0.3195,-0.6753,-0.6666,-0.5963,-0.6015,0.4911,0.4874,-0.2319,-0.0118,-0.0353,4.3094,-0.1187,-0.1250,0,1,0,0,1,0
1,0.2474,0,-0.3757,2.1586,-0.4000,-0.3889,-0.3446,-0.3127,-0.3195,1.4848,1.6005,0.5041,-0.6737,-0.6642,-0.6534,0.1158,-0.0854,-0.2882,-0.3152,-0.3228,-0.2932,0,1,0,0,1,1


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   limit_bal   24000 non-null  float64 
 1   sex_female  24000 non-null  int64   
 2   age         24000 non-null  float64 
 3   pay_1       24000 non-null  float64 
 4   pay_2       24000 non-null  float64 
 5   pay_3       24000 non-null  float64 
 6   pay_4       24000 non-null  float64 
 7   pay_5       24000 non-null  float64 
 8   pay_6       24000 non-null  float64 
 9   bill_amt1   24000 non-null  float64 
 10  bill_amt2   24000 non-null  float64 
 11  bill_amt3   24000 non-null  float64 
 12  bill_amt4   24000 non-null  float64 
 13  bill_amt5   24000 non-null  float64 
 14  bill_amt6   24000 non-null  float64 
 15  pay_amt1    24000 non-null  float64 
 16  pay_amt2    24000 non-null  float64 
 17  pay_amt3    24000 non-null  float64 
 18  pay_amt4    24000 non-null  float64 
 19  pay_

In [11]:
# Split target(y), features(X), train, test datasets

X_train = train_df.drop(target_col, axis=1)
y_train = train_df[target_col]

X_test = test_df.drop(target_col, axis=1)
y_test = test_df[target_col]

# Gaussian Naive Bayes (GNB)

In [12]:
class SimpleGaussianNB:
    '''
    Simplified Gaussian Naive Bayes Classifier
        
    Core Ideas:
    - Bayes' Theorem: P(y|x) ∝ P(y) * P(x|y)
    - Naive Assumption: Features are independent of each other
    - Gaussian Assumption: Each feature follows a normal distribution
        
    Therefore: P(x|y) = ∏ P(x_i|y) where P(x_i|y) ~ N(μ_yi, σ²_yi)
    '''
    def __init__(self):
        '''
        Initialization: Parameters to be stored after training
        '''
        self.classes = None # Class labels (e.g., [0, 1])
        self.class_priors = None # P(y): Prior probability of each class
        self.means = None # μ: Mean for each (class, feature) pair
        self.variances = None # σ²: Variance for each (class, feature) pair
        
    def fit(self, X, y):
        '''
        Train the model: Calculate mean, variance, and prior probability for each class
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Feature matrix of training data
        y : array-like, shape (n_samples,)
            Target labels of training data
            
        Returns:
        --------
        self : Trained model object
        
        Training Process:
        1. Separate data by each class
        2. Calculate prior for each class: P(y=c) = (# of samples in class c) / (total # of samples)
        3. Calculate mean (μ) and variance (σ²) for each (class, feature) pair
        '''
        # Convert DataFrame to numpy array (for consistency in internal calculations)
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values
        
        # Extract unique class labels (e.g., [0, 1])
        self.classes = np.unique(y)
        n_classes = len(self.classes) # Number of classes (2 for binary)
        n_features = X.shape[1] # Number of features
        
        # Initialize arrays to store training results
        # class_priors: shape (n_classes,) - one probability per class
        # means: shape (n_classes, n_features) - mean for each class and feature
        # variances: shape (n_classes, n_features) - variance for each class and feature
        self.class_priors = np.zeros(n_classes)
        self.means = np.zeros((n_classes, n_features))
        self.variances = np.zeros((n_classes, n_features))
        
        # Calculate statistics for each class
        for i, c in enumerate(self.classes):
            # Extract only samples belonging to current class c
            # X_c: shape (n_samples_in_class_c, n_features)
            X_c = X[y == c]
            
            # 1. Calculate prior probability: P(y=c)
            # Proportion of class c among all samples
            self.class_priors[i] = len(X_c) / len(X)
            
            # 2. Calculate mean for each feature: μ_c
            # X_c.mean(axis=0): Calculate mean for each column (feature)
            # Result: shape (n_features,) - one mean value per feature
            self.means[i] = X_c.mean(axis=0)
            
            # 3. Calculate variance for each feature: σ²_c
            # X_c.var(axis=0): Calculate variance for each column (feature)
            # Result: shape (n_features,) - one variance value per feature
            self.variances[i] = X_c.var(axis=0) + 1e-9 # + 1e-9: Prevent variance from becoming 0 (avoid division by zero later)
            
        return self
    
    def _gaussian_log_likelihood(self, x, mean, var):
        '''
        Calculate log-likelihood of Gaussian distribution for a single sample

        Logarithm
        - Multiplying very small probability values --> risk of underflow
        - In log space, multiplication becomes addition, which is numerically stable
        
        Parameters:
        -----------
        x : array-like, shape (n_features,)
            Feature vector of a single sample
        mean : array-like, shape (n_features,)
            Mean of each feature for a specific class
        var : array-like, shape (n_features,)
            Variance of each feature for a specific class
            
        Returns:
        --------
        log_likelihood : float
            log P(x|y) = sum over all log P(x_i|y)
        '''
        # Term 1: -0.5 * Σ log(2πσ²)
        # Calculate for each feature and sum all
        term1 = -0.5 * np.sum(np.log(2 * np.pi * var))
        
        # Term 2: -0.5 * Σ (x_i - μ)² / σ²
        # For each feature, divide (observed - mean)² by variance, then sum all
        term2 = -0.5 * np.sum((x - mean)**2 / var)
        
        return term1 + term2
    
    def predict(self, X):
        '''
        Predict classes for test data

        For each sample x:
        1. Calculate posterior probability for each class c
        2. Select the class with highest posterior probability
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Samples to predict
            
        Returns:
        --------
        predictions : array, shape (n_samples,)
            Predicted class for each sample
        '''
        # Convert DataFrame to numpy array
        if isinstance(X, pd.DataFrame):
            X = X.values
        
        predictions = []
        
        # Iterate through each sample (row) for prediction
        for x in X: # x: shape (n_features,) - single sample
            # Calculate posterior probability for each class
            posteriors = [] # Store log P(y|x) for each class
            
            # Calculate posterior probability for each class
            for i, c in enumerate(self.classes):
                # 1. Prior probability: log P(y=c)
                # self.class_priors[i] is a value between 0~1, so take log
                prior = np.log(self.class_priors[i])
                
                # 2. Likelihood: log P(x|y=c)
                # Probability that current sample x comes from distribution of class c
                # self.means[i]: Mean of each feature for class c (shape: n_features,)
                # self.variances[i]: Variance of each feature for class c (shape: n_features,)
                likelihood = self._gaussian_log_likelihood(
                    x, self.means[i], self.variances[i]
                )
                
                # 3. Posterior probability: log P(y=c|x) ∝ log P(y=c) + log P(x|y=c)
                # Log version of Bayes' Theorem
                # (Denominator P(x) is common to all classes, so can be omitted for comparison)
                posteriors.append(prior + likelihood) # e.g., list [np.float64(-77.65), np.float64(-53.45)]
            
            # 4. Select the class with highest posterior probability
            # np.argmax(posteriors): Returns index of maximum value
            # self.classes[index]: Actual class label at that index
            predictions.append(self.classes[np.argmax(posteriors)]) # -77.65 < -53.45 => index 1
        
        return np.array(predictions)
    
    def predict_proba(self, X):
        '''
        Return probability for each class
        
        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            Samples to predict
            
        Returns:
        --------
        probabilities : array, shape (n_samples, n_classes)
            Probability of each class for each sample
            e.g., [[0.8, 0.2], [0.3, 0.7], ...] (for binary classification)
            
        Probability Calculation Process:
        ---------------------------------
        1. Calculate log posterior probability for each class (same as predict)
        2. Convert to actual probabilities using Softmax transformation
           P(y=c|x) = exp(log P(y=c|x)) / Σ exp(log P(y=k|x))
        3. Use log-sum-exp trick for numerical stability
        '''
        if isinstance(X, pd.DataFrame):
            X = X.values
        
        probs = []
        
        # Iterate through each sample (row)
        for x in X: # x: shape (n_features,) - single sample
            posteriors = []
            
            # Calculate log posterior probability for each class (same as predict method)
            for i in range(len(self.classes)):
                prior = np.log(self.class_priors[i])
                likelihood = self._gaussian_log_likelihood(
                    x, self.means[i], self.variances[i]
                )
                posteriors.append(prior + likelihood)
                
            # Convert posteriors to numpy array
            # shape: (n_classes,) - log P(y|x) for each class
            posteriors = np.array(posteriors) # list -> ndarray object
            
            # Softmax transformation with log-sum-exp trick
            # -----------------------------------------------
            # Goal: P(y=c|x) = exp(log P(y=c|x)) / Σ exp(log P(y=k|x))
            # 
            # Problem: exp() can create very large/small values causing overflow/underflow
            # 
            # Solution: Log-sum-exp trick
            # 1. Subtract maximum value for stabilization: posteriors - max(posteriors)
            # This makes the largest value 0, and the rest negative
            # Mathematically equivalent: exp(a-M) / Σexp(b-M) = exp(a) / Σexp(b)
            posteriors = posteriors - np.max(posteriors)
            
            # 2. Apply exponential function
            # Now all values are ≤ 0, so no overflow
            exp_posteriors = np.exp(posteriors)
            
            # 3. Normalize: Divide each value by total sum to convert to probability
            # Sum of results = 1.0 (axiom of probability)
            probs.append(exp_posteriors / np.sum(exp_posteriors))
            
        # shape: (n_samples, n_classes)
        # Each row is one sample, each column is one class
        return np.array(probs)

In [17]:
# Make sure target is integer (0/1)
y_train_int = y_train.astype(int)
y_test_int = y_test.astype(int)

# GaussianNB built from scratch
model = SimpleGaussianNB()
model.fit(X_train, y_train) # train on integers

# Predict class labels (0 or 1)
my_preds = model.predict(X_test)

# Predict probability of default (class 1)
my_proba = model.predict_proba(X_test)
my_proba = my_proba[:, 1] # second column = positive class probability P(default)

print("GaussianNB (from scratch)")
print(f"Accuracy: {accuracy_score(y_test_int, my_preds):.4f}") # % of correct predictions (0 or 1)
print(f"F1 Score: {f1_score(y_test_int, my_preds):.4f}") # balance of precision & recall
print(f"ROC-AUC:  {roc_auc_score(y_test_int, my_proba):.4f}") # how well model ranks defaulters higher
print(f"PR-AUC:   {average_precision_score(y_test_int, my_proba):.4f}") # precision-focused AUC; strong on imbalanced data
print(f"Log Loss: {log_loss(y_test_int, my_proba):.4f}") # measures how confident and correct predictions are; penalty for wrong confidence

# sklearn GaussianNB (benchmark)
sklearn_gnb = GaussianNB()
sklearn_gnb.fit(X_train, y_train_int)

# Predict class labels and probability
sklearn_preds = sklearn_gnb.predict(X_test)
sklearn_proba = sklearn_gnb.predict_proba(X_test)[:, 1]

print("\nGaussianNB (sklearn)")
print(f"Accuracy: {accuracy_score(y_test_int, sklearn_preds):.4f}")
print(f"F1 Score: {f1_score(y_test_int, sklearn_preds):.4f}")
print(f"ROC-AUC:  {roc_auc_score(y_test_int, sklearn_proba):.4f}")
print(f"PR-AUC:   {average_precision_score(y_test_int, sklearn_proba):.4f}")
print(f"Log Loss: {log_loss(y_test_int, sklearn_proba):.4f}")

GaussianNB (from scratch)
Accuracy: 0.7713
F1 Score: 0.5206
ROC-AUC:  0.7471
PR-AUC:   0.4918
Log Loss: 1.6005

GaussianNB (sklearn)
Accuracy: 0.7713
F1 Score: 0.5206
ROC-AUC:  0.7471
PR-AUC:   0.4915
Log Loss: 1.6016
